In [ ]:
!pip install sagemaker boto3 pandas scikit-learn joblib

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import sagemaker
from sagemaker import get_execution_role
import boto3
import joblib
import argparse
import joblib

In [ ]:
df = pd.read_csv('Hotel Reservations.csv')

In [ ]:
def label_price(price):
    if price <= 85:
        return 1
    elif price < 115:
        return 2
    else:
        return 3

df['label_avg_price_per_room'] = df['avg_price_per_room'].apply(label_price)


In [ ]:
df.drop(columns=['avg_price_per_room'], inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

categorical_columns = df.select_dtypes(include=['object']).columns

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [ ]:
X = df.drop(columns=['label_avg_price_per_room'])
y = df['label_avg_price_per_room']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


In [ ]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
#Salva o modelo com JobLib
joblib_file = "random_forest_model.joblib"
joblib.dump(clf, joblib_file)

In [ ]:
#Carrega e testa o modelo salvo 
loaded_model = joblib.load(joblib_file)
y_pred_loaded = loaded_model.predict(X_test)
print(classification_report(y_test, y_pred_loaded))